# Meat Quality Assignment - Training our model

---

In the second step of our project, we train a binary classification model on the images to predict whether the meat is fresh or spoiled. Here we are not so interested in the the absolute performance of the model as much in defining a clear and reusable training pipeline for image data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from functools import partial

import numpy as np
import tensorflow as tf

from loguru import logger
from sklearn.model_selection import StratifiedShuffleSplit 

# Set parent folder as root to import local modules
module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path)

# Remove default logger and set level to INFO
logger.remove()
logger.add(sys.stderr, level="INFO")

from src.base.model import CNNModel
from src.base.pipelines import images_to_frame, decode_image, augment_image

In [ ]:
data_path = os.path.join(module_path, "data/meat-quality-assessment-based-on-deep-learning/")
output_path = os.path.join(module_path, "notebooks/output/training")
os.makedirs(output_path, exist_ok=True)

In [ ]:
df_img = images_to_frame(data_path=data_path)
df_img

In [ ]:
df_img.drop(columns=["label"], inplace=True)

In [ ]:
valid_size = 0.1
test_size = (df_img.shape[0] * valid_size) / (df_img.shape[0] - df_img.shape[0] * valid_size)
valid_split = StratifiedShuffleSplit(n_splits=1, test_size=valid_size, random_state=42)
test_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)

[(optim_idx, valid_idx)] = [i for i in valid_split.split(X=np.zeros([df_img.shape[0], 1]), y=df_img["target"], groups=None)]
[(train_idx, test_idx)] = [i for i in test_split.split(X=np.zeros([len(optim_idx), 1]), y=df_img.loc[optim_idx, "target"], groups=None)]

train_df = df_img.loc[train_idx, :]
test_df = df_img.loc[test_idx, :]
valid_df = df_img.loc[valid_idx, :]

logger.info(f"Training set size: {len(train_idx):,.0f}, test set size: {len(test_idx):,.0f}, validation set size: {len(valid_idx):,.0f}.")

In [ ]:
BATCH_SIZE = 32
RESIZE_SHAPE = (256, 256)
SHUFFLE_SIZE = 512
N_EPOCHS = 5
STEPS_PER_EPOCH = np.ceil(train_df.shape[0] / BATCH_SIZE)

In [ ]:
train_set = (
    tf.data.Dataset
    .from_tensor_slices((train_df["image_path"], train_df["target"]))
    .map(partial(decode_image, image_size=RESIZE_SHAPE, n_labels=2), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .repeat()
    .shuffle(512)
    .batch(32)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

valid_set = (
    tf.data.Dataset
    .from_tensor_slices((valid_df["image_path"], valid_df["target"]))
    .map(partial(decode_image, image_size=RESIZE_SHAPE, n_labels=2), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(32)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
for t, l in train_set.take(1):
    print(t.numpy().shape, l.numpy().shape)
    break

In [ ]:
model = CNNModel(width=4, height=4, name='CNNModel')
model.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
    metrics = [tf.keras.losses.BinaryCrossentropy(from_logits = True, name = 'BCE'), 'accuracy']
)
model.build(input_shape=(None, 256, 256, 3))

In [ ]:
model.summary()

In [ ]:
model.fit(  
    train_set, 
    epochs = N_EPOCHS,
    verbose = 1,
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_set
)